In [1]:
from project import *
from seeq import spy
import pandas as pd
import urllib.parse as urlparse
from urllib.parse import parse_qs
import ipywidgets as widgets
import qgrid
import re
from IPython.display import HTML, Javascript, display, clear_output
from ipywidgets import (
    GridspecLayout,
    Layout,
    Button,
    HBox,
    VBox,
    jslink,
    Box,
    AppLayout,
)


pd.set_option(
    "display.max_rows",
    None,  # display all rows
    "display.max_columns",
    6,  # display all columns
    "display.max_colwidth",
    None,  # expand column width
    "display.html.use_mathjax",
    False,
)  # disable Latex style mathjax rendering




In [2]:

#**********User Interface Functions**********

def pop_up(text):
    """shows error message

    Args:
        text (string): text for message to display
    """
    progressBar.layout.visibility = 'hidden'
    errorMessWidget.children[0].description = text
    errorMessWidget.layout.visibility = 'visible'
    loading_overlay(False)

def loading_overlay(disable):
    """disable or enable all screen inputs during loading. to prevent user form clicking buttons

    Args:
        disable (Bool):true to dissable all screen inputs, False to enable all screen inputs
    """
    grids = [push_grid, restore_grid]
    for item in grids:
        length = len(item.children)
        for q in range(length):
            item.children[q].disabled=disable
    
def updates_qgrid_widget_remove_row(b):
    updates_qgrid.remove_row(updates_qgrid.get_selected_rows())

def load_updates_qgrid(backups):
    """setting perameters adn build Qgrid for widgets to use
    Args:
        backups (_type_): bulkUploadUserData: DataFrame: Data to be put in qgrid
    Returns:
        _type_: Qgrid table
    """
    global updates_qgrid
    
    gridOps = {
        # SlickGrid options
        'fullWidthRows': True,
        'syncColumnCellResize': True,
        'forceFitColumns': False,
        #'defaultColumnWidth': 150,
        'rowHeight': 25,
        'enableColumnReorder': False,
        'enableTextSelectionOnCells': False,
        'editable': False,
        'autoEdit': False,
        'explicitInitialization': True,

        # Qgrid options
        'maxVisibleRows': 10,
        'minVisibleRows': 10,
        'sortable': True,
        'filterable': True,
        'highlightSelectedCell': False,
        'highlightSelectedRow': True,
        'resizable': True,
    }

    updates_qgrid = qgrid.show_grid(backups, show_toolbar=False, grid_options=gridOps,
                               column_definitions={ 
                                     'index': { 'maxWidth': 50, 'minWidth': 0, 'width': 50 }
                                    #  'Name': { 'maxWidth': None, 'minWidth': None, 'width': 280 }
                                    })
    return updates_qgrid

def load_restore_qgrid(backups):
    """setting perameters adn build Qgrid for widgets to use
    Args:
        backups (_type_): bulkUploadUserData: DataFrame: Data to be put in qgrid
    Returns:
        _type_: Qgrid table
    """
    global restore_qgrid
    
    gridOps = {
        # SlickGrid options
        'fullWidthRows': True,
        'syncColumnCellResize': True,
        'forceFitColumns': False,
        #'defaultColumnWidth': 150,
        'rowHeight': 25,
        'enableColumnReorder': False,
        'enableTextSelectionOnCells': False,
        'editable': False,
        'autoEdit': False,
        'explicitInitialization': True,

        # Qgrid options
        'maxVisibleRows': 10,
        'minVisibleRows': 10,
        'sortable': True,
        'filterable': True,
        'highlightSelectedCell': False,
        'highlightSelectedRow': True,
        'resizable': True,
    }

    restore_qgrid = qgrid.show_grid(backups, show_toolbar=False, grid_options=gridOps,
                               column_definitions={ 
                                    'index': { 'maxWidth': 50, 'minWidth': 0, 'width': 50 }
                                    # 'Name': { 'maxWidth': None, 'minWidth': None, 'width': 280 }
                                    })
    return restore_qgrid

def show_qgrid(qgrid_to_show, qgrid_load_function):
    widget_items = list(qgrid_to_show.children)
    widget_items[1] = qgrid_load_function
    qgrid_to_show.children = widget_items
    qgrid_to_show.layout.visibility = 'visible'


def appmode_load_wb():
    try:
        workbook_id = ''
        url = jupyter_notebook_url
        parsed = urlparse.urlparse(url)
        workbook_id = parse_qs(parsed.query)['workbookId'][0]

        if not workbook_id == '':
            #find and set Workbook Path Dropdown
            cur_wb_path = wb_paths_restore_cls.workbook_paths_df.loc[wb_paths_restore_cls.workbook_paths_df['ID'] == workbook_id, 'Path']
            if not cur_wb_path.empty:
                wb_path_dd.value = cur_wb_path.iloc[0]
            #set Selection Box
            found_wb = [item for item in workbooks_STB.options if item[1] == workbook_id]
            if len(found_wb) > 0:
                workbooks_STB.value=[found_wb[0][1]] 
    except:
        pass
    
#**********Form Click / on change Functions**********

#General Button / Click Functions
def update_wbpaths_butt_get(b):
    try:
        progressBar.layout.visibility = 'visible'
        progressBar.value = 1
        loading_overlay(True)
        wb_paths_from_file = False
        with progressOutput:
            wb_paths_cls.get_workbook_paths(wb_paths_from_file)
        progressOutput.clear_output()
        progressBar.value = 5
        #update Dropdown
        wb_path_dd.options = wb_paths_cls.uniq_paths_list
        
        #re pull workbooks for dropdown from file and update list
        wb_paths_restore_cls.get_workbook_paths(True)
        wb_path_restore_dd.options = wb_paths_restore_cls.uniq_paths_list
    except Exception as ex:
        pop_up(str(ex))
    finally:
        loading_overlay(False)
        progressBar.layout.visibility = 'hidden'
        push_updates_butt.disabled = True
        find_updates_butt.disabled = True
        update_wbpaths_butt.disabled = True

def error_Close_Button(b):
    errorMessWidget.layout.visibility = 'hidden'

def sucess_Close_Button(b):
    sucessMessWidget.layout.visibility = 'hidden'


#**********Update Button Click / On Change Functions********************
def show_workbook_change(b):
    """get pair of workbook names : ID associated with a given path

    Args:
        b (_type_): _description_
    """
    wb_names = wb_paths_cls.workbook_paths_df.loc[wb_paths_cls.workbook_paths_df['Path'].str.startswith(wb_path_dd.value), ['Name', 'ID']]
    workbooks_STB.options= list(zip(wb_names['Name'], wb_names['ID']))

def select_workbook_on_change(b):
    find_updates_butt.disabled=False

def find_updates_butt_get(b):
    """Function finds all changes between Test Manager and formula bar
    Args:
        b (_type_): _description_

    Raises:
        NoDataFound: _description_
        NoDataFound: _description_
    """
    try:
        global wb_signals_cls
        global fhr_formula_cls
        global main_signals_cls
        global middle_signals_cls
        global test_signals_cls
        global signal_updates

        loading_overlay(True)
        errors = True
        errorMessWidget.layout.visibility = 'hidden'
        sucessMessWidget.layout.visibility = "hidden"
        updates_qgrid_widget.layout.visibility = 'hidden'
        
        progressBar.layout.visibility = 'visible'
        progressBar.value = 1

        #run program to get updates for selected path
        # path = "FHR Alert Manager >> PB Folder >> Light Ends >> Gas Plants"
        #get selected workbooks IDs:
        workbook_ids_selected = list(workbooks_STB.value)

        if len(workbook_ids_selected) ==0:
            raise NoDataFound('Must Select Workbook') 

        #get Raw Workbook Signals
        wb_signals_cls = WorkBookSignals()
        progressBar.value = 3
        with progressOutput:
            wb_signals_cls.get_all_signals_by_wb_ids(workbook_ids_selected)
        progressOutput.clear_output()
        progressBar.value = 6
        
        with progressOutput:
            #get FHR Seeq API Formulas
            fhr_formula_cls= FhrFormulas()
        progressOutput.clear_output()
        progressBar.value = 7
        
        #get Main Signals
        main_signals_cls = MainSignals(wb_signals_cls.wb_signals_df)

        #get middle Signals
        middle_signals_cls = MiddleSignals(wb_signals_cls.wb_signals_df, fhr_formula_cls)
        
        #get test Signals
        test_signals_cls = TestSignals(wb_signals_cls.wb_signals_df, main_signals_cls.main_signals_df, middle_signals_cls.middle_signals_df, fhr_formula_cls)
        progressBar.value = 8
        
        if test_signals_cls.updated_test_signals.empty and middle_signals_cls.updated_middle_signals.empty:
            raise NoDataFound('No Changes Found')

        #updateSignals:
        signal_updates = Updates(wb_signals_cls)
        signal_updates.prep_signals_for_update(main_signals_cls, middle_signals_cls, test_signals_cls)
        # main_signals_cls
        progressBar.value = 10

        # show dataGrid
        data_to_show = signal_updates.combined_updates[['SignalName', 'ParameterUpdated', 'OriginalValue', 'ValueUpdateTo', 'Path', 'SignalID']]

        show_qgrid(updates_qgrid_widget, load_updates_qgrid(data_to_show))
        
        errors = False

    except Exception as ex:
        pop_up(str(ex))
    finally:
        loading_overlay(False)
        find_updates_butt.disabled=True
        progressBar.layout.visibility = 'hidden'
        if errors:
            push_updates_butt.disabled=True

def push_updates_get(b):
    """Function pushes user selected updates to seeq

    Args:
        b (_type_): _description_
    """
    try:
        loading_overlay(True)
        errorMessWidget.layout.visibility = 'hidden'
        sucessMessWidget.layout.visibility = "hidden"
        progressBar.layout.visibility = 'visible'
        progressBar.value = 1

        rows_to_update = signal_updates.combined_updates.loc[updates_qgrid.get_changed_df().index]
        progressBar.value = 5

        with progressOutput:
            signal_updates.push_user_selected_updates(rows_to_update)
        progressOutput.clear_output()


        updates_qgrid_widget.layout.visibility = 'hidden'
        
        progressBar.value = 10
        sucessMessWidget.layout.visibility = "visible"
        workbooks_STB.options = [""]
        
    except Exception as ex:
        pop_up(str(ex))
    finally:
        loading_overlay(False)
        progressBar.layout.visibility = 'hidden'
        find_updates_butt.disabled=True
        push_updates_butt.disabled=True


#********************Restore Button Click / on change Functions*******************
def show_workbook_restore_change(b):
    """Function updates drop down with backup files for selected workbook

    Args:
        b (_type_): _description_
    """
    #get pair of workbook names : ID associated with a given path
    wb_names = wb_paths_restore_cls.workbook_paths_df.loc[wb_paths_restore_cls.workbook_paths_df['Path'].str.startswith(wb_path_restore_dd.value), ['Name', 'ID']]
    workbooks_for_restore_dd.options= list(zip(wb_names['Name'], wb_names['ID']))
    push_restore_butt.disabled = True

def select_workbook_restore_on_change(b):
    show_restore_butt.disabled=False

def show_find_backup_change(b):
    show_restore_butt.disabled=False
    # restore_qgrid_widget.layout.visibility = 'hidden'

def show_backups_restore_change(b):
    #show backup file names  
    workbook_ids_selected = [workbooks_for_restore_dd.value]
    if len(workbook_ids_selected) > 0:
        restore_backups_cls = RestoreBackups(workbook_ids_selected) 
        if len(restore_backups_cls.backups_match) == 0:
            backups_for_wb_restore_dd.options = ['No Backups For Workbook']
            show_restore_butt.disabled = True
            push_restore_butt.disabled = True
        else:
            backups_for_wb_restore_dd.options = restore_backups_cls.backups_match
            show_restore_butt.disabled = False
            push_restore_butt.disabled = True


def show_restore_butt_get(b):
    """Functino shows data in a selected backup file

    Args:
        b (_type_): _description_

    Raises:
        NoDataFound: _description_
        NoDataFound: _description_
    """
    try:  
        global restore_backups_cls
        #Need to re initialize RestoreBackups class here, it keeps getting lost in show_backups_restore_change function.

        workbook_ids_selected = [workbooks_for_restore_dd.value]
        if len(workbook_ids_selected) == 0:
            raise NoDataFound('Must Select Workbook') 
        
        restore_backups_cls = RestoreBackups(workbook_ids_selected)
        if len(restore_backups_cls.backups_match) == 0:
            raise NoDataFound('No Signals to backup') 
        
        error_flag = True
        loading_overlay(True)
        progressBar.layout.visibility = 'visible'
        progressBar.value = 1
        #read in backup file
        restore_backups_cls.show_signals_in_backupfile([backups_for_wb_restore_dd.value])
        backup_signals = restore_backups_cls.restore_signals
        progressBar.value = 5
        show_qgrid(restore_qgrid_widget, load_restore_qgrid(backup_signals))
        push_restore_butt.disabled = False 
        progressBar.value = 10
        error_flag = False
    except Exception as ex:
        pop_up(str(ex))   
    finally:
        loading_overlay(False)
        progressBar.layout.visibility = 'hidden'
        show_restore_butt.disabled=True
        if error_flag:
            push_restore_butt.disabled=True

def push_restore_butt_get(b):
    """Function pushes backup values to seeq

    Args:
        b (_type_): button click event

    Raises:
        NoDataFound: restore class issue
    """
    try:
        loading_overlay(True)
        if restore_backups_cls.restore_signals.empty:
            raise NoDataFound('No Signals to backup')

        errorMessWidget.layout.visibility = 'hidden'
        sucessMessWidget.layout.visibility = "hidden"
        progressBar.layout.visibility = 'visible'
        progressBar.value = 5

        with progressOutput:
            restore_backups_cls.push_backup()
        progressOutput.clear_output()

        progressBar.value = 10
        sucessMessWidget.layout.visibility = "visible"

    except Exception as ex:
        pop_up(str(ex))
    finally:
        loading_overlay(False)
        progressBar.layout.visibility = 'hidden'
        restore_qgrid_widget.layout.visibility = 'hidden'
        show_restore_butt.disabled=True
        push_restore_butt.disabled=True



#User Interface Classes
class WorkBookPaths:
    def __init__(self):
        self.file_paths_csv = 'filepasdfasdfaths.csv'
        self.uniq_paths_list = []
        self.workbook_paths_df = pd.DataFrame()
    
    def get_workbook_paths(self, wb_paths_from_file):
        if wb_paths_from_file:
            self.workbook_path_unpkl()
        else:
            self.workbook_path_spy()
            self.workbook_path_pkl()

        self.workbook_path_uniq_list()

    def workbook_path_spy(self):
        self.workbook_paths_df = spy.workbooks.search(
            {
                "Path" : "Fasdfeswfdsfr"
                , "Workbook Type": "asdfasdf"
            }
            ,content_filter = 'all'
            ,recursive = True
            )
        if self.workbook_paths_df.empty:
            raise NoDataFound('No Workbook Paths found in shared Folder')
        
    def workbook_path_unpkl(self):
        self.workbook_paths_df = pd.read_csv(self.file_paths_csv)
        
    def workbook_path_pkl(self):
        self.workbook_paths_df.to_csv(self.file_paths_csv)

    def workbook_path_uniq_list(self):
        uniq_paths = self.workbook_paths_df['Path'].unique()
        uniq_paths.sort()
        self.uniq_paths_list = uniq_paths.tolist()


In [3]:
DBUGING_FLAG = True
#User Interface Setup and run
global wb_paths_cls
global wb_paths_restore_cls
# get workbook paths:
wb_paths_from_file = True
wb_paths_cls = WorkBookPaths()
wb_paths_cls.get_workbook_paths(wb_paths_from_file)

wb_paths_restore_cls = WorkBookPaths()
wb_paths_restore_cls.get_workbook_paths(wb_paths_from_file)


# *********************User Interface*********************
wb_path_dd = widgets.Dropdown(
    description="Select Workbook Path",
    options=wb_paths_cls.uniq_paths_list,
    disabled=False,
    # value="FHR Alert Manager >> PB Folder >> Light Ends >> Gas Plants",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto", align_items="stretch"),
)

wb_path_restore_dd = widgets.Dropdown(
    description="Select Workbook Path",
    options=wb_paths_restore_cls.uniq_paths_list,
    disabled=False,
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto", align_items="stretch"),
)

update_wbpaths_butt = widgets.Button(
    description="Update WB Paths",
    disabled=False,
    button_style="info",
    tooltip="Click me",
    icon="refresh",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto"),
)


workbooks_STB = widgets.SelectMultiple(
    options=[],
    description="WorkBooks:",
    description_tooltip="Multiple values can be selected with shift and/or ctrl",
    disabled=False,
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="100%"),
)


workbooks_for_restore_dd = widgets.Dropdown(
    description="Select WorkBook",
    options=[""],
    disabled=False,
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto", align_items="stretch"),
)


backups_for_wb_restore_dd = widgets.Dropdown(
    description="Select Backup File",
    options=[""],
    disabled=False,
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto", align_items="stretch"),
)


find_updates_butt = widgets.Button(
    description="Find Updates",
    disabled=False,
    button_style="info",
    tooltip="Click me",
    icon="search",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto"),
)

show_restore_butt = widgets.Button(
    description="Show Backup Signals",
    disabled=False,
    button_style="info",
    tooltip="Click me",
    icon="search",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto"),
)

push_updates_butt = widgets.Button(
    description="Push updates",
    disabled=False,
    button_style="info",
    tooltip="Click me",
    icon="send",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto"),
)

push_restore_butt = widgets.Button(
    description="Push Restore Signals",
    disabled=False,
    button_style="info",
    tooltip="Click me",
    icon="send",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto"),
)

qgrid_remove_row_butt = widgets.Button(
    description="Remove Row",
    disabled=False,
    tooltip="Click me",
    icon="remove",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto"),
)

updates_qgrid_widget = widgets.VBox(
    [qgrid_remove_row_butt, qgrid.show_grid(pd.DataFrame())]
)

restore_qgrid_widget = widgets.VBox(
    [widgets.Label(value=""), qgrid.show_grid(pd.DataFrame())]
)


# error message widget:
box_layout = Layout(
    display="flex", flex_flow="row", align_items="stretch", border="solid", width="100%"
)
errorWid = widgets.Valid(
    value=False,
    description="Valid!",
    style={"description_width": "initial"},
    disabled=False,
    layout=Layout(width="auto", height="auto", align_items="stretch"),
)
errorCloseButton = widgets.Button(
    description="Close",
    disabled=False,
    button_style="warning",
    tooltip="Click me",
    icon="times",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto", align_items="stretch"),
)
errorMessWidget = widgets.HBox([errorWid, errorCloseButton], layout=box_layout)

#Success message Widget
box_layout2 = Layout(
    display="flex", flex_flow="row", align_items="stretch", border="solid", width="100%"
)

if DBUGING_FLAG:
    success_widg_msg = "In Debug mode NO updates pushed to Seeq"
else:
    success_widg_msg = "Successfully Synced Test Manager"

successWid = widgets.Label(value=success_widg_msg)
successWidCloseButton = widgets.Button(
    description="Close",
    disabled=False,
    button_style="success",
    tooltip="Click me",
    icon="check",
    style={"description_width": "initial"},
    layout=Layout(width="auto", height="auto", align_items="stretch"),
)
sucessMessWidget = widgets.HBox([successWid, successWidCloseButton], layout=box_layout2)

#progress bar widget
progressOutput = widgets.Output(layout={"border": "1px solid black"})
progressBar = widgets.FloatProgress(
    value=0,
    min=0,
    max=10.0,
    description="Loading:",
    bar_style="info",
    style={"bar_color": "#ffff00"},
    orientation="horizontal",
)

#top notice bar setup:
headerWarningProgress = GridspecLayout(1, 3, height="50px")
headerWarningProgress[0:1, 0:1] = errorMessWidget
headerWarningProgress[0:1, 1:2] = sucessMessWidget
headerWarningProgress[0:1, 2:3] = progressBar


#Layout main grids:
gridNumOfRows = 15
gridNumofColumns = 10
push_grid = GridspecLayout(gridNumOfRows, gridNumofColumns, height="850px")
# manualGrid[RowStart:RowEnd, ColumnStart:ColumnEnd]
push_grid[0:1, 0:8] = wb_path_dd
push_grid[0:1, 8:9] = update_wbpaths_butt
push_grid[1:5, 0:8] = workbooks_STB
push_grid[6:7, 0:8] = find_updates_butt
push_grid[7:14, 0:10] = updates_qgrid_widget
push_grid[14:15, 0:8] = push_updates_butt

# Restore updates Grid
gridNumOfRows_restore = 15
gridNumofColumns_restore = 10
restore_grid = GridspecLayout(gridNumOfRows_restore, gridNumofColumns_restore, height="850px")
restore_grid[0:1, 0:8] = wb_path_restore_dd
restore_grid[0:1, 8:9] = update_wbpaths_butt
restore_grid[1:2, 0:8] = workbooks_for_restore_dd
restore_grid[2:3, 0:8] = backups_for_wb_restore_dd
restore_grid[3:4, 0:8] = show_restore_butt
restore_grid[4:13, 0:10] = restore_qgrid_widget
restore_grid[14:15, 0:8] = push_restore_butt

accordion = widgets.Accordion(children=[push_grid, restore_grid], selected_index=0)

#set initial states of widgets:
push_updates_butt.disabled = True
find_updates_butt.disabled = True
updates_qgrid_widget.layout.visibility = "hidden"

show_restore_butt.disabled = True
push_restore_butt.disabled = True
restore_qgrid_widget.layout.visibility = "hidden"

progressBar.layout.visibility = "hidden"
errorMessWidget.layout.visibility = "hidden"
sucessMessWidget.layout.visibility = "hidden"


mainForm = AppLayout(
    header=headerWarningProgress,
    left_sidebar=None,
    center=accordion,
    right_sidebar=None,
    footer=progressOutput,
    pane_heights=[0.2, 5, 0.5],
)

accordion.set_title(0, "Find Test Manager Mismatches")
accordion.set_title(1, "Restore Backups")

# *******FORM END***************

# *******Widget Event Functions*******************
wb_path_dd.observe(show_workbook_change)
workbooks_STB.observe(select_workbook_on_change)
find_updates_butt.on_click(find_updates_butt_get)
update_wbpaths_butt.on_click(update_wbpaths_butt_get)
push_updates_butt.on_click(push_updates_get)
errorCloseButton.on_click(error_Close_Button)
successWidCloseButton.on_click(sucess_Close_Button)
qgrid_remove_row_butt.on_click(updates_qgrid_widget_remove_row)

wb_path_restore_dd.observe(show_workbook_restore_change)
workbooks_for_restore_dd.observe(show_backups_restore_change)
backups_for_wb_restore_dd.observe(show_find_backup_change)
show_restore_butt.on_click(show_restore_butt_get)
push_restore_butt.on_click(push_restore_butt_get)

appmode_load_wb()

mainForm

AppLayout(children=(GridspecLayout(children=(HBox(children=(Valid(value=False, description='Valid!', layout=La…